In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import re
import random
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import word2vec, FastText
import gensim.downloader
from sklearn.decomposition import PCA

In [2]:
df=pd.read_csv('resume_entities.csv')

In [3]:
df['tag'].value_counts()

O                        54234
I-Skills                  2571
I-College Name             572
I-Degree                   407
I-Designation              384
B-Skills                   297
B-College Name             290
B-Degree                   279
B-Designation              244
I-Name                     226
B-Name                     215
B-Companies worked at      212
B-Location                 199
B-Email Address            189
B-Graduation Year          156
I-Companies worked at      139
I-Email Address             75
B-Years of Experience       32
I-Years of Experience       25
I-Location                   4
B-UNKNOWN                    1
Name: tag, dtype: int64

In [4]:
import pandas as pd

# Assuming 'df' is your DataFrame with 'token' and 'tag' columns

# Calculate the current value counts of 'O' tags
current_o_count = df['tag'].value_counts()['O']

if current_o_count > 3000:
    # Number of 'O' tags to remove to make their count equal to 3000
    tags_to_remove = current_o_count - 3000

    # Get the indices of 'O' tags
    o_indices = df[df['tag'] == 'O'].index

    # Convert the Int64Index to a list and then sample and remove excess 'O' tags
    indices_to_remove = o_indices.tolist()[:tags_to_remove]
    df_balanced = df.drop(indices_to_remove)
else:
    # No need to remove any 'O' tags, as their count is already less than or equal to 3000
    df_balanced = df.copy()


In [5]:
df=df_balanced
df['tag'].value_counts()

O                        3000
I-Skills                 2571
I-College Name            572
I-Degree                  407
I-Designation             384
B-Skills                  297
B-College Name            290
B-Degree                  279
B-Designation             244
I-Name                    226
B-Name                    215
B-Companies worked at     212
B-Location                199
B-Email Address           189
B-Graduation Year         156
I-Companies worked at     139
I-Email Address            75
B-Years of Experience      32
I-Years of Experience      25
I-Location                  4
B-UNKNOWN                   1
Name: tag, dtype: int64

In [6]:
# Assuming 'df' is your DataFrame with 'token' and 'tag' columns

# Calculate the maximum count among all tags
max_size = df['tag'].value_counts().max()

# Initialize an empty list to store the over-sampled DataFrames
lst = []

# Group the DataFrame by 'tag'
grouped = df.groupby('tag')

# Perform over-sampling for each group (tag)
for tag, group in grouped:
    if len(group) < max_size:
        lst.append(group.sample(max_size - len(group), replace=True))
        
# Concatenate the original DataFrame with the over-sampled DataFrames
df = pd.concat([df] + lst)

# Shuffle the rows of the balanced DataFrame to randomize the order
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now, 'balanced_df' contains the DataFrame with balanced class distribution

In [7]:
df['tag'].value_counts()

I-Email Address          3000
B-Degree                 3000
B-Years of Experience    3000
B-UNKNOWN                3000
I-Years of Experience    3000
B-Companies worked at    3000
B-Location               3000
I-Degree                 3000
B-Name                   3000
B-Designation            3000
B-Email Address          3000
I-Skills                 3000
O                        3000
B-College Name           3000
I-Name                   3000
I-Location               3000
B-Skills                 3000
B-Graduation Year        3000
I-College Name           3000
I-Companies worked at    3000
I-Designation            3000
Name: tag, dtype: int64

In [8]:
print('Number of sentences are '+str(df['sentence'].nunique()))
print('Number of Vocabs are '+str(df['token'].nunique()))

Number of sentences are 218
Number of Vocabs are 5084


In [9]:
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import gensim.downloader as api
from tensorflow.keras.layers import Embedding

tf.random.set_seed(123)

In [10]:
df['token']=df['token'].str.lower()

In [11]:
sentence_all = [(list(zip(group['token'], group['tag']))) for _, group in df.groupby('sentence')]

In [12]:
# Calculate the value counts for the "sentence" column
sentence_counts = df['sentence'].value_counts()

# Find the maximum value of the value counts
max_count = sentence_counts.max()
min_count = sentence_counts.min()

# Print the maximum value
print("Maximum value of sentence counts:", max_count)
print("Minimum value of sentence counts:", min_count)

Maximum value of sentence counts: 3324
Minimum value of sentence counts: 70


In [13]:
MAX_LEN = 3330
EMBEDDING = 300

In [14]:
words = list(set(df['token'].values))
tags = list(set(df["tag"].values))

In [15]:
#Creating word to index and tag to index
import sklearn

# Vocabulary Key:word -> Value:token_index
# The first 2 entries are reserved for PAD and UNK
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding

# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}

# Vocabulary Key:Label/Tag -> Value:tag_index
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0

# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}

In [16]:
import numpy as np
import pandas as pd

In [17]:
word2vec = {}  # Define the 'vocab' dictionary before the loop
with open('glove.840B.300d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        if len(values) < 2:
            continue
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
            word2vec[word] = coefs
        except ValueError:
            continue

In [18]:
# Embedding the word matrix, we will add this
hits = 0
misses = 0
# word2vec = api.load("word2vec-google-news-300")
embedding_dim = 300

# Prepare embedding matrix
embedding_matrix = np.zeros((len(word2idx), embedding_dim))
for word, i in zip(word2idx.keys(),word2idx.values()):
    embedding_vector = None
    try:
      embedding_vector = word2vec[word]
    except Exception :
      pass

    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 2619 words (1822 misses)


In [19]:
#Tagging word to index and tag to index

#from keras.preprocessing.sequence import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences

# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in sentence_all]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])

# Convert Tag/Label to tag_index
y = [[tag2idx[w[1]] for w in s] for s in sentence_all]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
from keras.utils import to_categorical

# One-Hot encode
y = [to_categorical(i, num_classes=len(tags)+1) for i in y]  # n_tags+1(PAD)
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
X_tr.shape, X_te.shape, np.array(y_tr).shape, np.array(y_te).shape


print('Raw Sample:\n ', ' '.join([w[0] for w in sentence_all[1]]))
print('\n ' )
print('Raw Label:\n ', ' '.join([w[1] for w in sentence_all[1]]))
print('\n ' )
print('After processing, sample:\n', X[0])
print('\n ' )
print('After processing, labels:\n', y[0])

Raw Sample:
  database ece jawaharlal microsoft 2014 kunam team 2014 bachelor technology indeed.com/r/ashok-kunam/7aac8767aacf10a0 2014 ashok kunam kunam indeed.com/r/ashok-kunam/7aac8767aacf10a0 2014 2014 ashok ece ashok kunam microsoft technology kunam 2014 microsoft indeed.com/r/ashok-kunam/7aac8767aacf10a0 lead 2014 database indeed.com/r/ashok-kunam/7aac8767aacf10a0 team nehru bachelor lead ashok jawaharlal team ashok database 2014 jawaharlal university kunam indeed.com/r/ashok-kunam/7aac8767aacf10a0 lead team lead 2014 microsoft 2014 microsoft database lead ashok 2014 2014 technology bachelor ashok 2014 ashok bachelor indeed.com/r/ashok-kunam/7aac8767aacf10a0 lead nehru ashok indeed.com/r/ashok-kunam/7aac8767aacf10a0 nehru indeed.com/r/ashok-kunam/7aac8767aacf10a0 indeed.com/r/ashok-kunam/7aac8767aacf10a0 technology indeed.com/r/ashok-kunam/7aac8767aacf10a0 university database university indeed.com/r/ashok-kunam/7aac8767aacf10a0 database 2014 technology 2014 university lead techno

In [20]:
len(word2idx)

4441

In [21]:
X_tr.shape, X_te.shape, np.array(y_tr).shape, np.array(y_te).shape

((196, 3330), (22, 3330), (196, 3330, 22), (22, 3330, 22))

In [22]:
y_tr = np.array(y_tr)
y_te = np.array(y_te)

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow_addons.utils.types import FloatTensorLike, TensorLike
from tensorflow_addons.layers import CRF
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow_addons.optimizers import AdamW



def build_model(max_len = MAX_LEN, input_dim = len(word2idx),embedding_dim = 300):
  # Model definition
  input = Input(shape=(max_len,))

  # Get embeddings
  embeddings = Embedding(
      input_dim,
      embedding_dim,
      weights=[embedding_matrix],
    input_length=max_len,
       mask_zero=True,
    trainable=True,
      name = 'embedding_layer'
  )(input)

  # variational biLSTM
  output_sequences = Bidirectional(LSTM(units=50, return_sequences=True))(embeddings)
  output_sequences = Bidirectional(LSTM(units=100, return_sequences=True))(output_sequences)
  # Stacking
  output_sequences = Bidirectional(LSTM(units=50, return_sequences=True))(output_sequences)

  # Adding more non-linearity
  dense_out = TimeDistributed(Dense(25, activation="relu"))(output_sequences)

  # CRF layer
  mask = Input(shape=(max_len,), dtype=tf.bool)
  crf = CRF(22, name='crf')
  predicted_sequence, potentials, sequence_length, crf_kernel = crf(dense_out)

  model = Model(input, potentials)
  model.compile(
      optimizer=AdamW(weight_decay=0.001),
      loss= SigmoidFocalCrossEntropy()) # Sigmoid focal cross entropy loss

  return model

model = build_model()

# Checkpointing
save_model = tf.keras.callbacks.ModelCheckpoint(filepath='ner_crf.h5',
  monitor='val_loss',
  save_weights_only=True,
  save_best_only=True,
  verbose=1
)

# Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=10)

callbacks = [save_model, es]

model.summary()

C:\Users\punna\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


NameError: name 'MAX_LEN' is not defined

In [1]:
history = model.fit(
  X_tr,
  y_tr,
  batch_size=64,
  epochs=20,
  validation_data=(X_te, y_te),
  callbacks=callbacks,
  shuffle=True
)

NameError: name 'model' is not defined